In [ ]:
import os
import time
import copy

import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import AutoAugment, AutoAugmentPolicy
from timm import create_model
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import os
import requests
import zipfile
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import defaultdict
import random
import medmnist

from torch.utils.data import Subset

import torch
import torch.nn as nn
import torch.optim as optim
from torch.linalg import inv
from torch.utils.data import DataLoader, Dataset, Subset
from torch.utils.data import TensorDataset
from torch.cuda.amp import autocast

from torchvision import transforms
from torchvision.transforms import AutoAugment, AutoAugmentPolicy
from PIL import Image

from transformers import ViTModel, ViTImageProcessor

import timm
from timm import create_model

from torch_geometric.nn import LabelPropagation
from torch_geometric.utils import dense_to_sparse

import numpy as np
import pandas as pd

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from torch_geometric.utils import to_networkx
from torch_geometric.data import Data

from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import rbf_kernel
from dppy.finite_dpps import FiniteDPP
from sklearn.metrics import pairwise_distances
from sklearn.manifold import TSNE

import os
import requests
import zipfile
from tqdm import tqdm
from collections import defaultdict, Counter
import random
import matplotlib.pyplot as plt
from torch.linalg import inv
import seaborn as sns
import medmnist

from sklearn.cluster import SpectralClustering
from sklearn.preprocessing import LabelEncoder

import community
import community.community_louvain as community
import networkx as nx
from sklearn.neighbors import kneighbors_graph

In [ ]:
vit_model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vit_model = vit_model.to(device)
vit_model.eval()

@torch.no_grad()    
def extract_features(loader, model, device):
    features = []
    labels = []
    for images, target in tqdm(loader, desc="Extracting Features"):
        images = images.to(device)
        target = target.to(device)
        output = model(images) # batch size, embedding size
        features.append(output)
        labels.append(target)
    features = torch.cat(features, dim=0)
    labels = torch.cat(labels, dim=0)
    return features, labels

train_loader, val_loader, test_loader = load_datasets(img_size, batch_size, shuffle_train=False)

train_features, train_labels = extract_features(train_loader, vit_model, device)
val_features, val_labels = extract_features(val_loader, vit_model, device)
test_features, test_labels = extract_features(test_loader, vit_model, device)

train_features.shape, val_features.shape, test_features.shape

torch.save(train_features, "vit_train_features.pth")
torch.save(train_labels, "vit_train_labels.pth")
torch.save(val_features, "vit_val_features.pth")
torch.save(val_labels, "vit_val_labels.pth")
torch.save(test_features, "vit_test_features.pth")
torch.save(test_labels, "vit_test_labels.pth")

In [ ]:
vgg_model = timm.create_model('vgg16', pretrained=True, num_classes=0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg_model = vgg_model.to(device)
vgg_model.eval()

train_loader, val_loader, test_loader = load_datasets(img_size, batch_size, shuffle_train=False)

train_features_vgg, train_labels_vgg = extract_features(train_loader, vgg_model, device)
val_features_vgg, val_labels_vgg = extract_features(val_loader, vgg_model, device)
test_features_vgg, test_labels_vgg = extract_features(test_loader, vgg_model, device)

torch.save(train_features_vgg, "vgg_train_features.pth")
torch.save(train_labels_vgg, "vgg_train_labels.pth")
torch.save(val_features_vgg, "vgg_val_features.pth")
torch.save(val_labels_vgg, "vgg_val_labels.pth")
torch.save(test_features_vgg, "vgg_test_features.pth")
torch.save(test_labels_vgg, "vgg_test_labels.pth")

In [ ]:
def random_coreset_selection(dataset, percentage):
    indices = []
    labels = dataset.labels.flatten()  # labels shape (N,)
    for cls in np.unique(labels):
        class_indices = np.where(labels == cls)[0]
        sample_count = max(1, int(len(class_indices) * (percentage / 100.0))) 
        selected = np.random.choice(class_indices, size=sample_count, replace=False)
        indices.extend(selected)
    return Subset(dataset, indices)


def load_datasubsets(img_size=img_size, batch_size=batch_size, coreset_percentage=None):
    """
    Loads datasets and, if coreset_percentage is provided, applies random coreset selection on the training set.
    """
    train_dataset = NumpyDataset(os.path.join(path, 'train_images.npy'),
                                 os.path.join(path, 'train_labels.npy'),
                                 train_transform)
    if coreset_percentage is not None:
        train_dataset = random_coreset_selection(train_dataset, coreset_percentage)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    val_dataset = NumpyDataset(os.path.join(path, 'val_images.npy'),
                               os.path.join(path, 'val_labels.npy'),
                               val_transform)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    test_dataset = NumpyDataset(os.path.join(path, 'test_images.npy'),
                                os.path.join(path, 'test_labels.npy'),
                                val_transform)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, test_loader

def compute_subset_class_counts(subset):
    labels = [int(subset.dataset.labels[i].item()) for i in subset.indices] # scalar for each label
    return dict(Counter(labels))

def compute_class_percentage(original_counts, subset_counts):
    percentages = {}
    for cls in original_counts:
        original_count = original_counts[cls]
        subset_count = subset_counts.get(cls, 0)
        percentages[cls] = (subset_count / original_count) * 100
    return percentages

percentages_list = [0.1, 1, 10]
for percentage in percentages_list:
    print(f"\n\n======================= PERCENTAGE {percentage}% =======================")
    train_loader, val_loader, test_loader = load_datasubsets(img_size, batch_size, percentage)
    
    train_class_counts = compute_class_counts(os.path.join(path, 'train_labels.npy'))
    train_subset_counts = compute_subset_class_counts(train_loader.dataset)
    train_subset_percentages = compute_class_percentage(train_class_counts, train_subset_counts)

    print('Train class counts:', train_class_counts)
    print('Train subset counts:', train_subset_counts)
    print('Train subset percentages:', train_subset_percentages)

In [ ]:
def train_model_subset(model, train_loader, val_loader, device, optimizer, scheduler, scaler, 
                       num_epochs=num_epochs, accumulation_steps=4, patience=5):
    metrics_log = {
        "train_loss": [],
        "train_accuracy": [],
        "train_f1": [],
        "val_loss": [],
        "val_accuracy": [],
        "val_f1": []
    }
    model.to(device)
    best_val_loss = float('inf')
    patience_counter = 0
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        all_labels, all_preds = [], []

        optimizer.zero_grad(set_to_none=True)

        for step, (images, labels) in enumerate(tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}")):
            images, labels = images.to(device), labels.to(device)
            with torch.cuda.amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, labels) / accumulation_steps

            scaler.scale(loss).backward()

            if (step + 1) % accumulation_steps == 0:
                try:
                    scaler.step(optimizer)
                except AssertionError:
                    optimizer.step()
                try:
                    scaler.update()
                except AssertionError:
                    pass
                optimizer.zero_grad(set_to_none=True)
                
            running_loss += loss.item() * accumulation_steps
            _, predicted = outputs.max(1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

        if (step + 1) % accumulation_steps != 0:
            try:
                scaler.step(optimizer)
            except AssertionError:
                optimizer.step()
            try:
                scaler.update()
            except AssertionError:
                pass
            optimizer.zero_grad(set_to_none=True)

        torch.cuda.empty_cache()

        train_accuracy = accuracy_score(all_labels, all_preds)
        train_f1 = f1_score(all_labels, all_preds, average='weighted')
        train_loss = running_loss / len(train_loader)
        
        model.eval()
        val_loss = 0.0
        val_labels, val_preds = [], []
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc="Validating"):
                images, labels = images.to(device), labels.to(device)
                with torch.cuda.amp.autocast():
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_labels.extend(labels.cpu().numpy())
                val_preds.extend(predicted.cpu().numpy())
        val_loss /= len(val_loader)
        val_accuracy = accuracy_score(val_labels, val_preds)
        val_f1 = f1_score(val_labels, val_preds, average='weighted')
        
        metrics_log["train_loss"].append(train_loss)
        metrics_log["train_accuracy"].append(train_accuracy)
        metrics_log["train_f1"].append(train_f1)
        metrics_log["val_loss"].append(val_loss)
        metrics_log["val_accuracy"].append(val_accuracy)
        metrics_log["val_f1"].append(val_f1)
        
        scheduler.step(val_loss)
    
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), "vgg_random_derma_best.pth")
            print("Validation loss improved. Model saved.")
        else:
            patience_counter += 1
            print(f"No improvement in validation loss for {patience_counter} epoch(s).")
        
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break
    
    print(f"Final Train Loss:     {metrics_log['train_loss'][-1]:.4f}")
    print(f"Final Train Accuracy: {metrics_log['train_accuracy'][-1]:.4f}")
    print(f"Final Train F1:       {metrics_log['train_f1'][-1]:.4f}")
    print(f"Final Val Loss:       {metrics_log['val_loss'][-1]:.4f}")
    print(f"Final Val Accuracy:   {metrics_log['val_accuracy'][-1]:.4f}")
    print(f"Final Val F1:         {metrics_log['val_f1'][-1]:.4f}")
    
    return metrics_log

def run_random_coreset_experiments(coreset_percentages=[0.1, 1, 10],
                                   seeds=[42, 43, 45],
                                   num_epochs=100,
                                   learning_rate=0.001,
                                   batch_size=128,
                                   img_size=224):
    overall_results = {}
    for cp in coreset_percentages:
        print(f"\n\n======================= Random Coreset Percentage: {cp}% =======================")
        results = []
        for seed in seeds:
            print(f"\n\n----------------------- SEED {seed} -----------------------")
            set_seed(seed)
            np.random.seed(seed)

            train_loader, val_loader, test_loader = load_datasubsets(img_size, batch_size, coreset_percentage=cp)
            
            model = timm.create_model('vgg16', pretrained=True, num_classes=num_classes)
            model = model.to(device)
            for param in model.parameters():
                param.requires_grad = True

            optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
            scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
            scaler = torch.cuda.amp.GradScaler()
            
            metrics_log = train_model_subset(model, train_loader, val_loader, device,
                                             optimizer, scheduler, scaler,
                                             num_epochs=num_epochs)
            
            final_val_acc = metrics_log["val_accuracy"][-1]
            final_val_f1  = metrics_log["val_f1"][-1]
            results.append((final_val_acc, final_val_f1))
        
        results = np.array(results)
        means = results.mean(axis=0)
        stds  = results.std(axis=0)
        print("\n\n======================= SUMMARY for {}% =======================".format(cp))
        print(f"Validation Accuracy: mean={means[0]:.4f}, std={stds[0]:.4f}")
        print(f"Validation F1:       mean={means[1]:.4f}, std={stds[1]:.4f}")
        overall_results[cp] = {"mean": means, "std": stds}
    return overall_results

In [ ]:
def kmeans_coreset_selection(features, labels, percentages, random_state=seed):
    if torch.is_tensor(features):
        features_np = features.cpu().numpy()
    else:
        features_np = np.array(features)
    if torch.is_tensor(labels):
        labels_np = labels.cpu().numpy()
    else:
        labels_np = np.array(labels)
    labels_np = labels_np.flatten() 

    random.seed(random_state)
    np.random.seed(random_state)

    unique_classes = np.unique(labels_np)
    dict_of_subsets = {}

    for p in percentages:
        selected_indices = []

        for c in unique_classes:
            class_indices = np.where(labels_np == c)[0]
            class_features = features_np[class_indices]
            n_samples = len(class_indices)


            k = int(np.floor((p / 100.0) * n_samples))
            k = max(1, min(k, n_samples))  # min 1

            if k > 1:
                kmeans = KMeans(n_clusters=k, random_state=random_state, n_init=10)
                kmeans.fit(class_features)
                centroids = kmeans.cluster_centers_
                labels_km = kmeans.labels_
            else:
                centroids = [np.mean(class_features, axis=0)]
                labels_km = np.zeros(n_samples, dtype=int)

            for cluster_id in range(k):
                cluster_member_indices = np.where(labels_km == cluster_id)[0]
                if len(cluster_member_indices) == 0:
                    continue  # Skip empty clusters.
                distances = np.linalg.norm(
                    class_features[cluster_member_indices] - centroids[cluster_id],
                    axis=1
                )
                rep_idx_in_cluster = cluster_member_indices[np.argmin(distances)]
                selected_indices.append(class_indices[rep_idx_in_cluster])

        while set(labels_np[selected_indices]) != set(unique_classes):
            missing_classes = set(unique_classes) - set(labels_np[selected_indices])
            for c in missing_classes:
                fallback_idx = np.random.choice(np.where(labels_np == c)[0]).item()
                selected_indices.append(fallback_idx)
        selected_indices = sorted(selected_indices)
        dict_of_subsets[p] = selected_indices

    return dict_of_subsets

In [ ]:
def run_kmeans_experiments_vgg_derma(kmeans_percentages=[0.1, 1, 10],
                           seeds=[42, 43, 45],
                           num_epochs=20,
                           learning_rate=0.0001,
                           batch_size=16,
                           img_size=224):
    overall_results = {}

    train_features = torch.load("vgg_derma_train_features.pth")
    train_labels = torch.load("vgg_derma_train_labels.pth")
    
    for kp in kmeans_percentages:
        print(f"\n\n======================= Kmeans Subset Percentage: {kp}% =======================")
        results = []
        for seed in seeds:
            print(f"\n\n----------------------- SEED {seed} -----------------------")
            set_seed(seed)
            np.random.seed(seed)

            selected_indices_dict = kmeans_coreset_selection(train_features, train_labels, [kp])
            selected_indices = selected_indices_dict[kp]
            print(f"Number of selected samples: {len(selected_indices)}")
            
            train_dataset = NumpyDataset(os.path.join(path, 'train_images.npy'),
                                         os.path.join(path, 'train_labels.npy'),
                                         train_transform)
            train_subset = Subset(train_dataset, selected_indices)
            train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
            

            val_dataset = NumpyDataset(os.path.join(path, 'val_images.npy'),
                                       os.path.join(path, 'val_labels.npy'),
                                       val_test_transform)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            
            model = timm.create_model('vgg16', pretrained=True, num_classes=num_classes)
            model = model.to(device)
            for param in model.parameters():
                param.requires_grad = True

            optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
            scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
            scaler = torch.cuda.amp.GradScaler()
            
            metrics_log = train_model_subset(model, train_loader, val_loader, device,
                                             optimizer, scheduler, scaler,
                                             num_epochs=num_epochs)
            final_val_acc = metrics_log["val_accuracy"][-1]
            final_val_f1  = metrics_log["val_f1"][-1]
            results.append((final_val_acc, final_val_f1))
        
        results = np.array(results)
        means = results.mean(axis=0)
        stds  = results.std(axis=0)
        print("\n\n======================= SUMMARY for {}% =======================".format(kp))
        print(f"Validation Accuracy: mean={means[0]:.4f}, std={stds[0]:.4f}")
        print(f"Validation F1:       mean={means[1]:.4f}, std={stds[1]:.4f}")
        overall_results[kp] = {"mean": means, "std": stds}
    return overall_results

In [ ]:
def compute_epsilon(full_features, subset_indices):
    """
    Computes the minimum distance of every full feature to the nearest coreset feature.
    Returns:
      - min_distances: array of minimum distances for each full feature.
      - epsilon: the maximum of these minimum distances (i.e. worst-case coverage).
    """
    subset_features = full_features[subset_indices]
    distances = pairwise_distances(full_features, subset_features) 
    min_distances = distances.min(axis=1) #for every train feature, min distance to any selected coreset sample
    epsilon = min_distances.max()
    return min_distances, epsilon

def plot_epsilon_distribution(full_features, subset_indices, method_name="", percentage=None):
    """
    Plots a histogram of the minimum distances from every point in the full set to the coreset.
    A vertical line indicates the epsilon (worst-case) distance.
    The title includes the coreset percentage and the selection method.
    """
    min_distances, epsilon = compute_epsilon(full_features, subset_indices)
    
    if percentage is not None:
        title_str = f"Min Distance Distribution ({method_name}, {percentage}% coreset)"
    else:
        title_str = f"Min Distance Distribution ({method_name})"
    
    plt.figure(figsize=(8, 6))
    plt.hist(min_distances, bins=50, alpha=0.7)
    plt.axvline(epsilon, color='red', linestyle='--', label=f"Epsilon = {epsilon:.2f}")
    plt.xlabel("Minimum distance to coreset")
    plt.ylabel("Count")
    plt.title(title_str)
    plt.legend()
    plt.show()
    return epsilon

def plot_2d_epsilon_net(full_features, subset_indices, method_name="", percentage=None):
    """
    Uses t-SNE to project the full features into 2D and plots all points (with low opacity)
    along with the selected coreset points (in red).
    The plot title includes the coreset percentage and the selection method.
    """
    tsne = TSNE(n_components=2, random_state=42)
    full_features_2d = tsne.fit_transform(full_features)
    subset_features_2d = full_features_2d[subset_indices]
    
    if percentage is not None:
        title_str = f"2D Visualization of Epsilon Net ({method_name}, {percentage}% coreset)"
    else:
        title_str = f"2D Visualization of Epsilon Net ({method_name})"
    
    plt.figure(figsize=(8, 6))
    plt.scatter(full_features_2d[:, 0], full_features_2d[:, 1], alpha=0.3, label="All Samples")
    plt.scatter(subset_features_2d[:, 0], subset_features_2d[:, 1], color="red", label="Coreset Samples")
    plt.title(title_str)
    plt.xlabel("t-SNE dimension 1")
    plt.ylabel("t-SNE dimension 2")
    plt.legend()
    plt.show()


for p in percentages:
    print(f"\n\n======================= PERCENTAGE {p}% =======================")
    full_train_images = np.load(os.path.join(path, 'train_images.npy'))  # shape: (N, H, W, C)
    full_train_features = full_train_images.reshape(full_train_images.shape[0], -1)
    
    train_dataset = NumpyDataset(
        os.path.join(path, 'train_images.npy'),
        os.path.join(path, 'train_labels.npy'),
        train_transform 
    )
    random_subset = random_coreset_selection(train_dataset, p)
    random_selected_indices = random_subset.indices  
    print(f"Number of random selected samples: {len(random_selected_indices)}")
    

    train_labels = np.load(os.path.join(path, 'train_labels.npy')).squeeze().astype(np.int64)
    N = full_train_images.shape[0]
    train_features_flat = full_train_images.reshape(N, -1)

    train_features_tensor = torch.tensor(train_features_flat, dtype=torch.float32)
    train_labels_tensor = torch.tensor(train_labels)
    
    kmeans_selected_indices_dict = kmeans_coreset_selection(train_features_tensor, train_labels_tensor, [p])
    kmeans_selected_indices = kmeans_selected_indices_dict[p]
    print(f"Number of kmeans selected samples: {len(kmeans_selected_indices)}")
    
    epsilon_random = plot_epsilon_distribution(full_train_features, random_selected_indices, method_name="Random Selection")
    plot_2d_epsilon_net(full_train_features, random_selected_indices, method_name="Random Selection")
    print(f"Random Selection Epsilon: {epsilon_random:.2f}")
    
    epsilon_kmeans = plot_epsilon_distribution(full_train_features, kmeans_selected_indices, method_name="Kmeans Selection")
    plot_2d_epsilon_net(full_train_features, kmeans_selected_indices, method_name="Kmeans Selection")
    print(f"Kmeans Selection Epsilon: {epsilon_kmeans:.2f}")

In [ ]:
def compute_embedding_propagation(features, alpha=0.5):
    features = features.cpu()
    N, d = features.shape
    squared_norms = (features ** 2).sum(dim=1, keepdim=True)  # (N, 1)
    d2 = squared_norms + squared_norms.t() - 2 * (features @ features.t())
    mask = torch.ones_like(d2, dtype=torch.bool)
    mask.fill_diagonal_(False)
    sigma2 = d2[mask].var()  
    A = torch.exp(-d2 / sigma2)
    A.fill_diagonal_(0) 
    D = A.sum(dim=1)  
    D_inv_sqrt = torch.diag(1.0 / torch.sqrt(D + 1e-8))
    L = D_inv_sqrt @ A @ D_inv_sqrt
    I = torch.eye(N)
    P = torch.inverse(I - alpha * L)
    propagated_features = P @ features
    return propagated_features

if __name__ == "__main__":
    train_feat_path = "vit_train_features.pth"
    val_feat_path = "vit_val_features.pth"
    test_feat_path = "vit_test_features.pth"
    
    vit_train_features = torch.load(train_feat_path)
    vit_val_features = torch.load(val_feat_path)
    vit_test_features = torch.load(test_feat_path)
    
    propagated_train_features = compute_embedding_propagation(vit_train_features, alpha=0.5)
    propagated_val_features = compute_embedding_propagation(vit_val_features, alpha=0.5)
    propagated_test_features = compute_embedding_propagation(vit_test_features, alpha=0.5)
    
    torch.save(propagated_train_features, "propagated_train_features.pth")
    torch.save(propagated_val_features, "propagated_val_features.pth")
    torch.save(propagated_test_features, "propagated_test_features.pth")

In [ ]:
def embedding_propagation_coreset_selection_for_dataset(dataset, propagated_features, labels, percentage, alpha=0.5):
    selected_indices = []
    propagated_features = propagated_features.cpu()
    labels = labels.cpu()
    unique_labels = torch.unique(labels)
    
    for cls in unique_labels:
        class_mask = (labels == cls)
        class_indices = torch.nonzero(class_mask, as_tuple=True)[0]
        X_c = propagated_features[class_indices]
        n_c, d = X_c.shape
        
        centroid = X_c.mean(dim=0, keepdim=True)
        centroid_sim = F.cosine_similarity(X_c, centroid, dim=-1)
        sorted_order = torch.argsort(centroid_sim, descending=True)
        num_to_select = max(1, int(n_c * (percentage / 100.0)))
        selected_class_indices = class_indices[sorted_order[:num_to_select]]
        selected_indices.append(selected_class_indices)
    
    selected_indices = torch.cat(selected_indices)
    return Subset(dataset, selected_indices.cpu().numpy())

def load_datasubsets_embedding_propagation(img_size=224, batch_size=64, coreset_percentage=None, 
                                            propagated_features=None, vit_labels=None):
    train_dataset = NumpyDataset(
        os.path.join(path, 'train_images.npy'),
        os.path.join(path, 'train_labels.npy'),
        transform=train_transform
    )
    if coreset_percentage is not None and propagated_features is not None and vit_labels is not None:
        train_dataset = embedding_propagation_coreset_selection_for_dataset(
            train_dataset, propagated_features, vit_labels, coreset_percentage
        )
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    val_dataset = NumpyDataset(
        os.path.join(path, 'val_images.npy'),
        os.path.join(path, 'val_labels.npy'),
        transform=val_test_transform
    )
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    test_dataset = NumpyDataset(
        os.path.join(path, 'test_images.npy'),
        os.path.join(path, 'test_labels.npy'),
        transform=val_test_transform
    )
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader


def run_embedding_propagation_coreset_vgg_derma(coreset_percentages=[0.1, 1, 10],
                                                  seeds=[42, 43, 45],
                                                  num_epochs=20,
                                                  learning_rate=0.0001,
                                                  batch_size=16,
                                                  img_size=224):
    propagated_train_features = torch.load("propagated_vgg_derma_train_features.pth")
    vit_train_labels = torch.from_numpy(np.load(os.path.join(path, 'train_labels.npy'))).long()
    
    overall_results = {}
    for cp in coreset_percentages:
        print(f"\n\n======================= Coreset Percentage: {cp}% =======================")
        results = []
        for seed in seeds:
            print(f"\n\n----------------------- SEED {seed} -----------------------")
            set_seed(seed)
            train_loader, val_loader, test_loader = load_datasubsets_embedding_propagation(
                img_size=img_size,
                batch_size=batch_size,
                coreset_percentage=cp,
                propagated_features=propagated_train_features,
                vit_labels=vit_train_labels
            )
            
            model = timm.create_model('vgg16', pretrained=True, num_classes=num_classes)
            model = model.to(device)
            for param in model.parameters():
                param.requires_grad = True

            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                                   factor=0.1, patience=5, verbose=True)
            scaler = torch.cuda.amp.GradScaler()
            
            metrics_log = train_model_subset(model, train_loader, val_loader, device,
                                             optimizer, scheduler, scaler,
                                             num_epochs=num_epochs)
            
            final_val_acc = metrics_log["val_accuracy"][-1]
            final_val_f1  = metrics_log["val_f1"][-1]
            results.append((final_val_acc, final_val_f1))
        
        results = np.array(results)
        means = results.mean(axis=0)
        stds  = results.std(axis=0)
        print("\n\n======================= SUMMARY for {}% =======================".format(cp))
        print(f"Validation Accuracy: mean={means[0]:.4f}, std={stds[0]:.4f}")
        print(f"Validation F1:       mean={means[1]:.4f}, std={stds[1]:.4f}")
        overall_results[cp] = {"mean": means, "std": stds}
    return overall_results


def evaluate_propagation_quality(original_features, propagated_features):
    original_features = original_features.cpu()
    propagated_features = propagated_features.cpu()
    
    cosine_sim = F.cosine_similarity(original_features, propagated_features, dim=-1)
    
    mean_sim = cosine_sim.mean().item()
    std_sim = cosine_sim.std().item()
    min_sim = cosine_sim.min().item()
    max_sim = cosine_sim.max().item()
    
    print("Propagation Quality Evaluation:")
    print(f"Mean cosine similarity: {mean_sim:.4f}")
    print(f"Std of cosine similarity: {std_sim:.4f}")
    print(f"Min cosine similarity: {min_sim:.4f}")
    print(f"Max cosine similarity: {max_sim:.4f}")
    
    plt.figure(figsize=(8, 5))
    plt.hist(cosine_sim.numpy(), bins=50, alpha=0.75, color='skyblue', edgecolor='black')
    plt.title("Histogram of Cosine Similarities\nbetween Original and Propagated Embeddings")
    plt.xlabel("Cosine Similarity")
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.show()

evaluate_propagation_quality(vit_train_features, propagated_train_features)

In [ ]:
def compute_affinity_in_chunks(features, chunk_size=500):
    N, d = features.shape
    affinity = torch.empty((N, N), dtype=features.dtype)
    for i in range(0, N, chunk_size):
        end_i = min(i + chunk_size, N)
        aff_chunk = F.cosine_similarity(features[i:end_i].unsqueeze(1),
                                        features.unsqueeze(0), dim=-1)
        aff_chunk = (aff_chunk + 1) / 2 # A_ij = cos(zi,zj) + 1 / 2
        affinity[i:end_i] = aff_chunk
    return affinity

def compute_spectral_embedding(features, n_components=10, chunk_size=1000):
    features = features.cpu()
    N, d = features.shape
    affinity = compute_affinity_in_chunks(features, chunk_size=chunk_size)
    affinity.fill_diagonal_(0)  
    D = affinity.sum(dim=1)
    D_inv_sqrt = torch.diag(1.0 / torch.sqrt(D + 1e-8))
    L = D_inv_sqrt @ affinity @ D_inv_sqrt  
    eigvals, eigvecs = torch.linalg.eig(L)
    eigvals = eigvals.real
    eigvecs = eigvecs.real
    sorted_indices = torch.argsort(eigvals, descending=True)
    spectral_embedding = eigvecs[:, sorted_indices[:n_components]]
    
    return spectral_embedding

if __name__ == "__main__":
    train_feat_path = "vit_train_features.pth"
    val_feat_path   = "vit_val_features.pth"
    test_feat_path  = "vit_test_features.pth"
    
    vit_train_features = torch.load(train_feat_path)
    vit_val_features   = torch.load(val_feat_path)
    vit_test_features  = torch.load(test_feat_path)
    
    spectral_train_embeddings = compute_spectral_embedding(vit_train_features, n_components=10)
    spectral_val_embeddings   = compute_spectral_embedding(vit_val_features, n_components=10)
    spectral_test_embeddings  = compute_spectral_embedding(vit_test_features, n_components=10)
    
    torch.save(spectral_train_embeddings, "spectral_train_embeddings.pth")
    torch.save(spectral_val_embeddings,   "spectral_val_embeddings.pth")
    torch.save(spectral_test_embeddings,  "spectral_test_embeddings.pth")

In [ ]:
def supervised_spectral_coreset_selection_with_spectral_emb(spectral_embeddings, labels, percentage, random_state=42):
    selected_indices = []
    spectral_embeddings_np = spectral_embeddings.cpu().numpy()
    labels_np = labels.cpu().numpy()
    unique_labels = np.unique(labels_np)
    
    for cls in unique_labels:
        cls_idx = np.where(labels_np == cls)[0]
        X_cls = spectral_embeddings_np[cls_idx]  
        n_cls = X_cls.shape[0]
        n_sel = max(1, int(n_cls * (percentage / 100.0)))
        
        if n_sel >= n_cls:
            selected_indices.extend(cls_idx.tolist())
        elif n_sel == 1:
            sim_matrix = cosine_similarity(X_cls)
            avg_sim = sim_matrix.mean(axis=1)
            medoid_idx = np.argmax(avg_sim)
            selected_indices.append(cls_idx[medoid_idx])
        else:
            kmeans = KMeans(n_clusters=n_sel, random_state=random_state)
            cluster_labels = kmeans.fit_predict(X_cls)
            for cluster in range(n_sel):
                cluster_mask = (cluster_labels == cluster)
                if np.sum(cluster_mask) == 0:
                    continue
                cluster_indices = np.where(cluster_mask)[0]
                X_cluster = X_cls[cluster_indices]
                sim_cluster = cosine_similarity(X_cluster)
                avg_sim_cluster = sim_cluster.mean(axis=1)
                medoid_in_cluster = cluster_indices[np.argmax(avg_sim_cluster)]
                selected_indices.append(cls_idx[medoid_in_cluster])
    
    return np.array(selected_indices)


def supervised_spectral_coreset_selection_for_dataset(dataset, spectral_embeddings, labels, percentage, random_state=42):
    selected_idx = supervised_spectral_coreset_selection_with_spectral_emb(spectral_embeddings, labels, percentage, random_state)
    return Subset(dataset, selected_idx)

def load_datasubsets_spectral(img_size=224, batch_size=64, coreset_percentage=None, 
                               spectral_embeddings=None, vit_labels=None):
    """
    Loads the train/val/test datasets and applies spectral coreset selection on the training set using
    pre-computed spectral embeddings.
    """
    train_dataset = NumpyDataset(
        os.path.join(path, 'train_images.npy'),
        os.path.join(path, 'train_labels.npy'),
        transform=train_transform
    )
    if coreset_percentage is not None and spectral_embeddings is not None and vit_labels is not None:
        train_dataset = supervised_spectral_coreset_selection_for_dataset(
            train_dataset, spectral_embeddings, vit_labels, coreset_percentage, random_state=42
        )
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    val_dataset = NumpyDataset(
        os.path.join(path, 'val_images.npy'),
        os.path.join(path, 'val_labels.npy'),
        transform=val_test_transform
    )
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    test_dataset = NumpyDataset(
        os.path.join(path, 'test_images.npy'),
        os.path.join(path, 'test_labels.npy'),
        transform=val_test_transform
    )
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, test_loader

def run_spectral_coreset_experiments_derma_vgg(coreset_percentages=[0.1, 1, 10],
                                     seeds=[42, 43, 45],
                                     num_epochs=20,
                                     learning_rate=0.0001,
                                     batch_size=16,
                                     img_size=224):
    """
    Run experiments on coreset selection using pre-computed spectral embeddings.
    """
    spectral_train_embeddings = torch.load("spectral_vgg_derma_train_embeddings.pth")
    vit_train_labels = torch.from_numpy(np.load(os.path.join(path, 'train_labels.npy'))).long()
    
    overall_results = {}
    for cp in coreset_percentages:
        print(f"\n\n======================= Coreset Percentage: {cp}% =======================")
        results = []
        for seed in seeds:
            print(f"\n\n----------------------- SEED {seed} -----------------------")
            set_seed(seed)
            train_loader, val_loader, test_loader = load_datasubsets_spectral(
                img_size=img_size,
                batch_size=batch_size,
                coreset_percentage=cp,
                spectral_embeddings=spectral_train_embeddings,
                vit_labels=vit_train_labels
            )
            
            model = timm.create_model('vgg16', pretrained=True, num_classes=num_classes)
            model = model.to(device)
            for param in model.parameters():
                param.requires_grad = True

            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                                   factor=0.1, patience=5, verbose=True)
            scaler = torch.cuda.amp.GradScaler()
            
            metrics_log = train_model_subset(model, train_loader, val_loader, device,
                                             optimizer, scheduler, scaler,
                                             num_epochs=num_epochs)
            
            final_val_acc = metrics_log["val_accuracy"][-1]
            final_val_f1  = metrics_log["val_f1"][-1]
            results.append((final_val_acc, final_val_f1))
        
        results = np.array(results)
        means = results.mean(axis=0)
        stds  = results.std(axis=0)
        print(f"\n\n======================= SUMMARY for {cp}% =======================")
        print(f"Validation Accuracy: mean={means[0]:.4f}, std={stds[0]:.4f}")
        print(f"Validation F1:       mean={means[1]:.4f}, std={stds[1]:.4f}")
        overall_results[cp] = {"mean": means, "std": stds}
    return overall_results